<a href="https://colab.research.google.com/github/erendagasan/Eren-Dagasan-Personal/blob/main/Stcok_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
!pip install bta-lib
!pip install ta
!pip install yfinance

xu30 = ["AKBNK", "AKSEN", "ALARK", "ARCLK", "ASELS", "BIMAS", "EKGYO", "ENKAI", "EREGL", "FROTO", "GARAN", "GUBRF", "HEKTS","KRDMD","KCHOL", "KOZAL", "KOZAA", "ODAS", "PGSUS", "SAHOL",
        "SASA", "PETKM", "SISE", "TAVHL", "THYAO", "TUPRS", "TOASO", "TCELL", "ISCTR", "YKBNK"]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [25]:
import yfinance as yf
import btalib
import numpy as np
import pandas as pd
from ta.trend import PSARIndicator
from ta.momentum import WilliamsRIndicator
from ta.trend import AroonIndicator

start="2022-01-01"
end="2023-06-20"
stock = "ASELS.IS"
data = yf.download(stock, start=start, end=end)
data.tail()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2023-06-07,53.349998,54.700001,53.000000,54.299999,54.299999,80506726
2023-06-08,54.049999,54.849998,52.950001,54.599998,54.599998,56424253
2023-06-09,55.000000,55.900002,53.900002,55.599998,55.599998,53273425
2023-06-12,56.000000,56.599998,54.049999,54.150002,54.150002,45106943
2023-06-14,52.000000,53.500000,51.000000,52.250000,52.250000,42624485


In [36]:
# Below 30 is oversold, upper 70 is overbought
data["RSI"] = btalib.rsi(data["Close"], period=14).df

#If little period conflicts with the bigger one it means buy else sell.
data["SMA9"] = btalib.sma(data['Close'], period=9).df
data["SMA20"] = btalib.sma(data['Close'], period=20).df

#A reading below 20 generally represents an oversold market and a reading above 80 an overbought market. Look only STOCH-D.
data["STOCH-K"] = btalib.stoch(data['High'], data['Low'], data['Close']).df["k"]
data["STOCH-D"] = btalib.stoch(data['High'], data['Low'], data['Close']).df["d"]

#When the MACD line crosses above the signal line, it generates a buy signal. This indicates a potential bullish trend reversal.
#Additionally, monitor the MACD histogram. When the histogram bars turn positive (change from negative to positive), it confirms the buy signal and suggests increasing buying pressure.
data["MACD"] = btalib.macd(data['Close']).df["macd"]
data["SIGNAL"] = btalib.macd(data['Close']).df["signal"]
data["HISTOGRAM"] = btalib.macd(data['Close']).df["histogram"]

#Buy Signal: When the price of the stock touches or crosses below the lower Bollinger Band, it can be considered a buy signal.
#This suggests that the stock may be oversold, and a potential price rebound is expected.
#Sell Signal: When the price of the stock touches or crosses above the upper Bollinger Band, it can be considered a sell signal.
#This indicates that the stock may be overbought, and a potential price correction is expected.
data["BB-UPPER"] = btalib.bbands(data['Close']).df['top']
data["BB-MID"] = btalib.bbands(data['Close']).df['mid']
data["BB-LOWER"] = btalib.bbands(data['Close']).df['bot']

#When the current price of the stock falls below a certain number of standard deviations (GET THE MIN AND MAX VALUE OF THE COLUMN AND MAKE IT THE THRESHOLD)
#from the mean, it generates a buy signal. This suggests that the price has deviated significantly from the average and may present a buying opportunity.
data["STDEV"] = data["Close"].rolling(window=10).std()

#Generate a buy signal when the price crosses above the PSAR.
#This indicates a potential reversal in the downward trend and suggests a buying opportunity.
data["PSAR"] = PSARIndicator(data["High"], data["Low"], data["Close"]).psar()

#Williams
#Use the Williams Percent Range (%R) formula to calculate the %R values based on historical price data.
#The %R values range from -100 to 0, where values close to -100 indicate oversold conditions and values close to 0 indicate overbought conditions.
#Generate a buy signal when the %R value crosses above a certain threshold (e.g., -80) from below.
#This indicates a potential reversal from oversold conditions and suggests a buying opportunity.
data["WILLIAMS"] = WilliamsRIndicator(data["High"], data["Low"], data["Close"]).williams_r()

#The difference between these two lines indicates whether there is overbought (a positive number) or oversold (a negative number).
data["AROON"] = AroonIndicator(close=data["Close"], window=25).aroon_indicator()

In [27]:
data = data.dropna()
data = data.reset_index()
data.tail()

In [28]:
scores = []
for row in range(data.shape[0]):
  score = 0

  if data["RSI"][row] < 30:
    score += 1
  else:
    score+=(100-int(data["RSI"][row]))/70

  if data["SMA9"][row] > data["SMA20"][row] and data["Close"][row] > data["SMA9"][row]:
    score += 1
  else:
    score += -1

  if data["STOCH-D"][row] <= 20:
    score += 1
  elif data["STOCH-D"][row] >= 80:
    score += -1

  #TODO: Additionally, monitor the MACD histogram. When the histogram bars turn positive (change from negative to positive), it confirms the buy signal and suggests increasing buying pressure.
  if data["MACD"][row] > data["SIGNAL"][row]:
    score+=1
  elif data["MACD"][row] < data["SIGNAL"][row]:
    score+=-1

  if data["BB-LOWER"][row] > data["Close"][row]:
    score += 1
  elif data["BB-MID"][row] < data["Close"][row]:
    score += +0.25
  elif data["BB-MID"][row] > data["Close"][row]:
    score += -0.25
  else:
    score += -1

  if (data["STDEV"].max() + data["STDEV"].min()) / 2 > data["STDEV"][row]:
    score+=1
  else:
    score+=-1

  if data["PSAR"][row] < data["Close"][row]:
    score+=1
  else:
    score+=-1

  #-20 as the overbought threshold and -80 as the oversold threshold.
  if data["WILLIAMS"][row] > -20:
    score+=-1
  elif data["WILLIAMS"][row] < -80:
    score+=1

  if data["AROON"][row] > 0:
    score+=-1
  else:
    score+=1

  scores.append(score)

data["SCORE"] = scores

<ipython-input-28-278048d08f38>:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["SCORE"] = scores


In [32]:
output_file = stock + ":" + start + ":" + end + '.xlsx'
writer = pd.ExcelWriter(output_file)
data.to_excel(writer, index=False, sheet_name=stock)
writer.save()

<ipython-input-32-f67379de50e6>:4: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()


In [33]:
tr_balance = 2000
stock_balance = 0

transactions = []

for row in range(data.shape[0]):
  if data["SCORE"][row] >= 3.5 and tr_balance > 100:
   stock_balance = tr_balance / data["Close"][row]
   tr_balance-= stock_balance*data["Close"][row]
   transactions.append(["BUY", data.index[row], data["Close"][row], stock_balance, tr_balance])

  elif data["SCORE"][row] <= -3 and stock_balance > 0:
    tr_balance+=stock_balance*data["Close"][row]
    stock_balance-=stock_balance
    transactions.append(["SELL", data.index[row], data["Close"][row], stock_balance, tr_balance])


print(stock_balance)
tr_balance

93.24936430216697


0.0

In [34]:
transactions_df = pd.DataFrame(transactions, columns=["B/S", "DATE", "PRICE", "STOCK BALANCE", "TRY BALANCE"])
transactions_df

,B/S,DATE,PRICE,STOCK BALANCE,TRY BALANCE
0,BUY,0,21.700001,92.165895,0.000000
1,SELL,31,24.780001,0.000000,2283.870951
2,BUY,66,23.980000,95.240659,0.000000
3,SELL,159,32.779999,0.000000,3121.988674
4,BUY,165,33.480000,93.249364,0.000000


In [96]:
def create_indicators(data):
  # Below 30 is oversold, upper 70 is overbought
  data["RSI"] = btalib.rsi(data["Close"], period=14).df

  #If little period conflicts with the bigger one it means buy else sell.
  data["SMA9"] = btalib.sma(data['Close'], period=9).df
  data["SMA20"] = btalib.sma(data['Close'], period=20).df

  #A reading below 20 generally represents an oversold market and a reading above 80 an overbought market. Look only STOCH-D.
  data["STOCH-K"] = btalib.stoch(data['High'], data['Low'], data['Close']).df["k"]
  data["STOCH-D"] = btalib.stoch(data['High'], data['Low'], data['Close']).df["d"]

  #When the MACD line crosses above the signal line, it generates a buy signal. This indicates a potential bullish trend reversal.
  #Additionally, monitor the MACD histogram. When the histogram bars turn positive (change from negative to positive), it confirms the buy signal and suggests increasing buying pressure.
  data["MACD"] = btalib.macd(data['Close']).df["macd"]
  data["SIGNAL"] = btalib.macd(data['Close']).df["signal"]
  data["HISTOGRAM"] = btalib.macd(data['Close']).df["histogram"]

  #Buy Signal: When the price of the stock touches or crosses below the lower Bollinger Band, it can be considered a buy signal.
  #This suggests that the stock may be oversold, and a potential price rebound is expected.
  #Sell Signal: When the price of the stock touches or crosses above the upper Bollinger Band, it can be considered a sell signal.
  #This indicates that the stock may be overbought, and a potential price correction is expected.
  data["BB-UPPER"] = btalib.bbands(data['Close']).df['top']
  data["BB-MID"] = btalib.bbands(data['Close']).df['mid']
  data["BB-LOWER"] = btalib.bbands(data['Close']).df['bot']

  #When the current price of the stock falls below a certain number of standard deviations (GET THE MIN AND MAX VALUE OF THE COLUMN AND MAKE IT THE THRESHOLD)
  #from the mean, it generates a buy signal. This suggests that the price has deviated significantly from the average and may present a buying opportunity.
  data["STDEV"] = data["Close"].rolling(window=10).std()

  #Generate a buy signal when the price crosses above the PSAR.
  #This indicates a potential reversal in the downward trend and suggests a buying opportunity.
  data["PSAR"] = PSARIndicator(data["High"], data["Low"], data["Close"]).psar()

  #Williams
  #Use the Williams Percent Range (%R) formula to calculate the %R values based on historical price data.
  #The %R values range from -100 to 0, where values close to -100 indicate oversold conditions and values close to 0 indicate overbought conditions.
  #Generate a buy signal when the %R value crosses above a certain threshold (e.g., -80) from below.
  #This indicates a potential reversal from oversold conditions and suggests a buying opportunity.
  data["WILLIAMS"] = WilliamsRIndicator(data["High"], data["Low"], data["Close"]).williams_r()

  #The difference between these two lines indicates whether there is overbought (a positive number) or oversold (a negative number).
  data["AROON"] = AroonIndicator(close=data["Close"], window=25).aroon_indicator()

  data = data.dropna()
  data = data.reset_index()

  return data

def score(data):
  scores = []

  for row in range(data.shape[0]):
    score = 0

    if data["RSI"][row] < 30:
      score += 1
    else:
      score+=(100-int(data["RSI"][row]))/70

    if data["SMA9"][row] > data["SMA20"][row] and data["Close"][row] > data["SMA9"][row]:
      score += 1
    else:
      score += -1

    if data["STOCH-D"][row] <= 20:
      score += 1
    elif data["STOCH-D"][row] >= 80:
      score += -1

    #TODO: Additionally, monitor the MACD histogram. When the histogram bars turn positive (change from negative to positive), it confirms the buy signal and suggests increasing buying pressure.
    if data["MACD"][row] > data["SIGNAL"][row]:
      score+=1
    elif data["MACD"][row] < data["SIGNAL"][row]:
      score+=-1

    if data["BB-LOWER"][row] > data["Close"][row]:
      score += 1
    elif data["BB-MID"][row] < data["Close"][row]:
      score += +0.25
    elif data["BB-MID"][row] > data["Close"][row]:
      score += -0.25
    else:
      score += -1

    if (data["STDEV"].max() + data["STDEV"].min()) / 2 > data["STDEV"][row]:
      score+=1
    else:
      score+=-1

    if data["PSAR"][row] < data["Close"][row]:
      score+=1
    else:
      score+=-1

    #-20 as the overbought threshold and -80 as the oversold threshold.
    if data["WILLIAMS"][row] > -20:
      score+=-1
    elif data["WILLIAMS"][row] < -80:
      score+=1

    if data["AROON"][row] > 0:
      score+=-1
    else:
      score+=1

    scores.append(score)

  data["SCORE"] = scores

  return data

def save_to_excel(stock, data):
  output_file = stock + ":" + start + ":" + end + '.xlsx'
  writer = pd.ExcelWriter(output_file)
  data.to_excel(writer, index=True, sheet_name=stock)
  writer.save()

def simulation(stock, data, balance):
  tr_balance = balance
  stock_balance = 0

  transactions = []

  for row in range(data.shape[0]):
    if data["SCORE"][row] >= 3.5 and tr_balance > 100:
      stock_balance = tr_balance / data["Close"][row]
      tr_balance-= stock_balance*data["Close"][row]
      transactions.append([stock, "BUY", data.index[row], data["Close"][row], stock_balance, tr_balance])

    elif data["SCORE"][row] <= -3 and stock_balance > 0:
      tr_balance+=stock_balance*data["Close"][row]
      stock_balance-=stock_balance
      transactions.append([stock,"SELL", data.index[row], data["Close"][row], stock_balance, tr_balance])

  transactions_df = pd.DataFrame(transactions, columns=["STOCK","B/S", "DATE", "PRICE", "STOCK BALANCE", "TRY BALANCE"])
  return transactions_df


In [97]:
start="2022-01-01"
end="2023-06-30"

buy_stocks = []
sell_stocks = []

simulation_stocks = pd.DataFrame()

for stock in xu30:
  data = yf.download(stock+".IS", start=start, end=end)
  data = create_indicators(data)
  data = score(data)
  # save_to_excel(stock, data)


  if data.tail(1)["SCORE"].item() > 5 or 3 <= data.tail(1)["SCORE"].item() <= 5:
    buy_stocks.append([data.tail(1)["Date"].item(), stock, data.tail(1)["Close"].item(), data.tail(1)["SCORE"].item()])
    simulation_df = simulation(stock, data, balance=2000)
    simulation_stocks = pd.concat([simulation_stocks, simulation_df])

  elif data.tail(1)["SCORE"].item() < -2.5:
    sell_stocks.append([data.tail(1)["Date"].item(), stock, data.tail(1)["Close"].item(), data.tail(1)["SCORE"].item()])

[*********************100%***********************]  1 of 1 completed


/usr/local/lib/python3.10/dist-packages/btalib/meta/lines.py:302: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  trailer = trailprefix.append(series[p2:])
/usr/local/lib/python3.10/dist-packages/btalib/meta/lines.py:302: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  trailer = trailprefix.append(series[p2:])
/usr/local/lib/python3.10/dist-packages/btalib/meta/lines.py:302: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  trailer = trailprefix.append(series[p2:])
/usr/local/lib/python3.10/dist-packages/btalib/meta/lines.py:302: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  trailer = trailprefix.append(series[p2:])
/usr/local/lib/python3.1

[*********************100%***********************]  1 of 1 completed


/usr/local/lib/python3.10/dist-packages/btalib/meta/lines.py:302: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  trailer = trailprefix.append(series[p2:])
/usr/local/lib/python3.10/dist-packages/btalib/meta/lines.py:302: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  trailer = trailprefix.append(series[p2:])
/usr/local/lib/python3.10/dist-packages/btalib/meta/lines.py:302: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  trailer = trailprefix.append(series[p2:])
/usr/local/lib/python3.10/dist-packages/btalib/meta/lines.py:302: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  trailer = trailprefix.append(series[p2:])
/usr/local/lib/python3.1

[*********************100%***********************]  1 of 1 completed


...

/usr/local/lib/python3.10/dist-packages/btalib/meta/lines.py:302: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  trailer = trailprefix.append(series[p2:])
/usr/local/lib/python3.10/dist-packages/btalib/meta/lines.py:302: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  trailer = trailprefix.append(series[p2:])
/usr/local/lib/python3.10/dist-packages/btalib/meta/lines.py:302: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  trailer = trailprefix.append(series[p2:])
/usr/local/lib/python3.10/dist-packages/btalib/meta/lines.py:302: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  trailer = trailprefix.append(series[p2:])
/usr/local/lib/python3.1

[*********************100%***********************]  1 of 1 completed


/usr/local/lib/python3.10/dist-packages/btalib/meta/lines.py:302: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  trailer = trailprefix.append(series[p2:])
/usr/local/lib/python3.10/dist-packages/btalib/meta/lines.py:302: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  trailer = trailprefix.append(series[p2:])
/usr/local/lib/python3.10/dist-packages/btalib/meta/lines.py:302: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  trailer = trailprefix.append(series[p2:])
/usr/local/lib/python3.10/dist-packages/btalib/meta/lines.py:302: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  trailer = trailprefix.append(series[p2:])
/usr/local/lib/python3.1

[*********************100%***********************]  1 of 1 completed


/usr/local/lib/python3.10/dist-packages/btalib/meta/lines.py:302: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  trailer = trailprefix.append(series[p2:])
/usr/local/lib/python3.10/dist-packages/btalib/meta/lines.py:302: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  trailer = trailprefix.append(series[p2:])
/usr/local/lib/python3.10/dist-packages/btalib/meta/lines.py:302: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  trailer = trailprefix.append(series[p2:])
/usr/local/lib/python3.10/dist-packages/btalib/meta/lines.py:302: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  trailer = trailprefix.append(series[p2:])
/usr/local/lib/python3.1

In [98]:
buy_stocks = pd.DataFrame(buy_stocks, columns=["Date","Stock", "Price","Score"])
buy_stocks

,Date,Stock,Price,Score
0,2023-06-15,AKBNK,18.650000,3.807143
1,2023-06-15,BIMAS,161.300003,4.821429
2,2023-06-15,GARAN,29.900000,4.850000
3,2023-06-15,ISCTR,13.480000,3.778571


In [99]:
sell_stocks = pd.DataFrame(sell_stocks, columns=["Date","Stock", "Price","Score"])
sell_stocks

,Date,Stock,Price,Score
0,2023-06-15,KOZAA,43.220001,-2.535714
1,2023-06-15,SASA,54.900002,-2.521429
2,2023-06-15,TAVHL,86.099998,-3.178571


In [100]:
simulation_stocks

,STOCK,B/S,DATE,PRICE,STOCK BALANCE,TRY BALANCE
0,AKBNK,BUY,22,7.510000,266.311576,0.000000e+00
1,AKBNK,SELL,141,14.190000,0.000000,3.778961e+03
2,AKBNK,BUY,160,13.060000,289.353831,0.000000e+00
3,AKBNK,SELL,310,15.580000,0.000000,4.508133e+03
4,AKBNK,BUY,320,16.340000,275.895510,0.000000e+00
0,BIMAS,BUY,98,83.800003,23.866348,0.000000e+00
1,BIMAS,SELL,293,158.500000,0.000000,3.782816e+03
2,BIMAS,BUY,328,161.199997,23.466602,0.000000e+00
0,GARAN,BUY,12,11.750000,170.212766,0.000000e+00
1,GARAN,SELL,58,14.980000,0.000000,2.549787e+03
